In [ ]:
#data
docs = ['Where are you REALLY from?',
       'Go back to where you came from!',
       'What\'s my Chinese name?',
       'Are you from North or South Korea?',
       'They\'re so pretty for a black person',
       'Scientifically speaking, whites are superior to blacks',
       'Filipinos, Mexicans, they\'re all the same!',
       'I bet I know Japanese better than you lol',
       'You\re such a banana',
       'You don\'t act black at all, in fact I just think of you as white!',
       'Go find a god damned job and learn to speak English or I\'ll blind you with a piece of dental floss, you slanty-eyed mother fucker',
        'Y\'all are like the good kind of black people!',
       'That Jew belongs in an oven.',
        'She needs to get her Jap ass out.',
       'I love you so much, you\'re so cool!!!',
       'She\'s such a hard worker, and I really appreciate that.',
       'The state of this nation is a travesty and something needs to change.',
       'Obama was the greatest president this country has seen.',
       'Machine learning is a method of data analysis that automates analytical model building. It is a branch of artificial intelligence based on the idea that systems can learn from data, identify patterns and make decisions with minimal human intervention.',
       'I\'d like a nice bowl of ramen to hit the spot right now!',
       'Capitalism is an unsustainable and unjust idea, and we need to make changes before global warming destroys the world.',
       'The cutest member of BTS has to be V, but if we\'re going off sense of humor, Jin. However, if we\'re considering intellect, RM. Not that you asked but the best dancer is JHOPE and the guy I can\'t believe is my age is Jungkook.',
       'For people who have red-green colorblindness, the colors red and green look exactly the same.',
       'We hope to see you again!']

#labels
labels = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0])

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np

#hyperparameters
max_features = 1000
max_length = 140

#split the data into train, test, and validation sets
train = pd.read_csv('labeled_data.csv')
train_portion = train.shape[0]*0.64
test_portion = train_portion + train.shape[0]*0.2

x_train = train.ix[0:train_portion,6].values
y_train = train.ix[0:train_portion,5].values

x_test = train.ix[train_portion:test_portion, 6].values
y_test = train.ix[train_portion:test_portion, 5].values

x_valid = train.ix[test_portion:,6].values
y_valid = train.ix[test_portion:,5].values

#encode and pad
encoded_train = [one_hot(word, max_features) for word in x_train]
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')

encoded_test = [one_hot(word, max_features) for word in x_test]
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

#shape
model = Sequential()
model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

#compile
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#fit
model.fit(padded_train, y_train, batch_size=48, epochs=10)
score = model.evaluate(padded_test, y_test, batch_size=48)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for la

Epoch 1/10
15862/15862 [==============================] - 51s 3ms/step - loss: -1.5932 - acc: 0.7647
Epoch 2/10
15862/15862 [==============================] - 54s 3ms/step - loss: -1.6340 - acc: 0.7659
Epoch 3/10
15862/15862 [==============================] - 52s 3ms/step - loss: -1.6342 - acc: 0.7659
Epoch 4/10
15862/15862 [==============================] - 50s 3ms/step - loss: -1.6341 - acc: 0.7659
Epoch 5/10
15862/15862 [==============================] - 49s 3ms/step - loss: -1.6342 - acc: 0.7659
Epoch 6/10
15862/15862 [==============================] - 53s 3ms/step - loss: -1.6342 - acc: 0.7659
Epoch 7/10
15862/15862 [==============================] - 58s 4ms/step - loss: -1.6342 - acc: 0.7659
Epoch 8/10
15862/15862 [==============================] - 54s 3ms/step - loss: -1.6342 - acc: 0.7659
Epoch 9/10
15862/15862 [==============================] - 57s 4ms/step - loss: -1.6342 - acc: 0.7659
Epoch 10/10
4956/4956 [==============================] - 6s 1ms/step


In [14]:
print(score)

[-2.1809800137811557, 0.7857142845597164]
